In [1]:
# Libraries
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, log_loss, roc_curve, recall_score, confusion_matrix, classification_report, roc_auc_score, precision_recall_curve, auc
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
import xgboost as xgb
from sklearn.dummy import DummyClassifier

from datasets import load_dataset

/Users/mounir/Desktop/PROJETS/Statapp/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the dataset
data = load_dataset('criteo/Fairjob')
df = data['train'].to_pandas()

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Utils

def click_rank_utility(df, y_pred):
    """
    Implémente la fonction Click-Rank Utility U(f).
    """
    df_test = df.loc[y_pred.index].copy()
    df_test = df_test.assign(pred=y_pred)

    # Evite le biais de position
    df_test = df_test[df_test['displayrandom'] == 1].copy()

    # Trier les produits affichés dans chaque impression par score de prédiction croissant
    df_test = df_test.sort_values(by=['impression_id', 'pred'], ascending=[True, True])  

    # Calcul du rank
    df_test['rank_order'] = df_test.groupby('impression_id')['pred'].rank(method='min', ascending=True)

    # I(Y_D = 1) * rank_J f(X_J, D)
    df_test['utility'] = df_test['click'] * df_test['rank_order']

    # Moyenne de U(f) sur chaque impression’
    click_rank_per_impression = df_test.groupby('impression_id')['utility'].mean()

    # Moyenne finale sur toutes les bannières
    return click_rank_per_impression.mean()

def nllh(y_true, y_pred):
    y_pred_np = np.array(y_pred) + 1e-15
    y_true_np = np.array(y_true)
    
    epsilon = 1e-15

    y_pred_clip = np.clip(y_pred_np, epsilon, 1 - epsilon)

    log_loss_np = y_true_np * np.log(y_pred_clip) + (1 - y_true_np) * np.log(1 - y_pred_clip)
    return -np.mean(log_loss_np)

def DP(df, y_pred : pd.Series):

    df_test = df.loc[y_pred.index].copy()
    df_test['y_pred'] = y_pred

    df_senior = df_test [df_test['senior'] == 1]
    df_senior_male = df_senior[df_senior['protected_attribute'] == 1]
    df_senior_female = df_senior[df_senior['protected_attribute'] == 0]
    return np.mean(df_senior_male['y_pred']) - np.mean(df_senior_female['y_pred'])

def AUC(y_true, y_pred_prob):
    fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
    return auc(fpr, tpr)

def report_results(df, y_pred_prob, y_pred):

    if not isinstance(y_pred, pd.Series):
        raise ValueError("y_pred must be a pandas Series")
    
    df_test = df.loc[y_pred.index].copy()
    y_true = df_test['click']

    click_rank_value = click_rank_utility(df, y_pred)
    nllh_value = nllh(y_true, y_pred_prob)
    auc_value = AUC(y_true, y_pred_prob)
    dp_value = DP(df, y_pred)

    return {
    'Click Rank Utility': round(click_rank_value, 5),
    'Negative Log-Likelihood': round(nllh_value, 5),
    'AUC': round(auc_value, 5),
    'Demographic Parity': round(dp_value, 5)
}


In [10]:
#Test des metriques

# Séparer les features et la target
X = df.drop(columns=["click"])
y = df["click"]

# Splitter les données en train/test
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir les données au format DMatrix (format optimisé pour XGBoost)
dtrain1 = xgb.DMatrix(X_train1, label=y_train1)
dtest1 = xgb.DMatrix(X_test1, label=y_test1)

# Définir les paramètres du modèle
params = {
    'objective': 'binary:logistic',  # Classification binaire
    'eval_metric': 'auc',            # Utiliser l'AUC comme métrique
    'max_depth': 6,                  # Profondeur maximale des arbres
    'eta': 0.3,                      # Taux d'apprentissage
    'subsample': 0.8,                # Sous-échantillonnage des lignes
    'colsample_bytree': 0.8          # Sous-échantillonnage des colonnes
}

# Entraîner le modèle XGBoost
model1 = xgb.train(params, dtrain1, num_boost_round=100)

# Générer les prédictions de probabilités
y_pred_prob1 = model1.predict(dtest1)

# Générer les prédictions binaires avec un seuil de 0.5
y_pred1 = (y_pred_prob1 >= 0.5).astype(int)

# Transformer les prédictions en Series avec les bons index
y_pred_prob1 = pd.Series(y_pred_prob1, index=X_test1.index)
y_pred1 = pd.Series(y_pred1, index=X_test1.index)

# Afficher les métriques
report_results(df, y_pred_prob1, y_pred1)


{'Click Rank Utility': np.float64(0.00853),
 'Negative Log-Likelihood': np.float64(0.03439),
 'AUC': np.float64(0.83964),
 'Demographic Parity': np.float64(-0.00023)}

In [11]:
# Entrainement d'un XGBoost unfair

import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
import pandas as pd
import numpy as np


X = df.drop(columns=["click"])
y = df["click"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    """ Fonction d'optimisation pour choisir les meilleurs hyperparamètres """
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'objective': 'binary:logistic',
        'eval_metric': 'auc'
    }
    
    model = xgb.XGBClassifier(**params)
    score = cross_val_score(model, X_train, y_train, scoring='roc_auc', cv=3).mean()

    #print(f"Trial {trial.number}: AUC = {score:.5f}, Params = {params}")
    
    return score

# Effectuer N essais pour le tuning
N_trials = 50
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=N_trials)

best_params = study.best_params
print("Best parameters:", best_params)

# Entraîner le modèle final avec les meilleurs paramètres
best_model = xgb.XGBClassifier(**best_params)
best_model.fit(X_train, y_train)

y_pred_prob1 = best_model.predict_proba(X_test)[:, 1]
y_pred1 = (y_pred_prob1 >= 0.5).astype(int)

# Transformer les prédictions en Series avec les bons index
y_pred_prob1 = pd.Series(y_pred_prob1, index=X_test.index)
y_pred1 = pd.Series(y_pred1, index=X_test.index)

# Afficher les métriques
xgb_unfair_results = report_results(df, y_pred_prob1, y_pred1)
xgb_unfair_results

[I 2025-02-12 15:33:34,626] A new study created in memory with name: no-name-3a6234bb-ba9a-45e6-a354-8d23df0420cd
[I 2025-02-12 15:33:57,787] Trial 0 finished with value: 0.8079871086135838 and parameters: {'max_depth': 9, 'subsample': 0.7934362249624916, 'n_estimators': 129, 'colsample_bytree': 0.8846381704202704}. Best is trial 0 with value: 0.8079871086135838.


Trial 0: AUC = 0.80799, Params = {'max_depth': 9, 'subsample': 0.7934362249624916, 'n_estimators': 129, 'colsample_bytree': 0.8846381704202704, 'objective': 'binary:logistic', 'eval_metric': 'auc'}


[I 2025-02-12 15:34:33,102] Trial 1 finished with value: 0.7970314490310524 and parameters: {'max_depth': 8, 'subsample': 0.7009062773765784, 'n_estimators': 235, 'colsample_bytree': 0.9477720814985668}. Best is trial 0 with value: 0.8079871086135838.


Trial 1: AUC = 0.79703, Params = {'max_depth': 8, 'subsample': 0.7009062773765784, 'n_estimators': 235, 'colsample_bytree': 0.9477720814985668, 'objective': 'binary:logistic', 'eval_metric': 'auc'}


[I 2025-02-12 15:34:46,997] Trial 2 finished with value: 0.8421657757049799 and parameters: {'max_depth': 4, 'subsample': 0.9610563947950373, 'n_estimators': 151, 'colsample_bytree': 0.8050024121346935}. Best is trial 2 with value: 0.8421657757049799.


...

[I 2025-02-12 15:46:18,173] Trial 48 finished with value: 0.8071458633389823 and parameters: {'max_depth': 9, 'subsample': 0.903880387340334, 'n_estimators': 295, 'colsample_bytree': 0.5294987272359146}. Best is trial 29 with value: 0.8465797598569944.


Trial 48: AUC = 0.80715, Params = {'max_depth': 9, 'subsample': 0.903880387340334, 'n_estimators': 295, 'colsample_bytree': 0.5294987272359146, 'objective': 'binary:logistic', 'eval_metric': 'auc'}


[I 2025-02-12 15:46:28,568] Trial 49 finished with value: 0.8446809974344692 and parameters: {'max_depth': 4, 'subsample': 0.9368256264447123, 'n_estimators': 102, 'colsample_bytree': 0.9693561061306252}. Best is trial 29 with value: 0.8465797598569944.


Trial 49: AUC = 0.84468, Params = {'max_depth': 4, 'subsample': 0.9368256264447123, 'n_estimators': 102, 'colsample_bytree': 0.9693561061306252, 'objective': 'binary:logistic', 'eval_metric': 'auc'}
Best parameters: {'max_depth': 3, 'subsample': 0.865015359929133, 'n_estimators': 121, 'colsample_bytree': 0.6055094773262627}


{'Click Rank Utility': np.float64(0.00853),
 'Negative Log-Likelihood': np.float64(0.03394),
 'AUC': np.float64(0.844),
 'Demographic Parity': np.float64(-2e-05)}

In [18]:
# Dummy Classifier

from sklearn.dummy import DummyClassifier

X = df.drop(columns=["click", 'protected_attribute'])
y = df["click"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner un DummyClassifier qui prédit toujours la classe majoritaire
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)

y_pred_prob_dummy = dummy_clf.predict_proba(X_test)[:, 1]
y_pred_dummy = dummy_clf.predict(X_test)

# Transformer les prédictions en Series avec les bons index
y_pred_prob_dummy = pd.Series(y_pred_prob_dummy, index=X_test.index)
y_pred_dummy = pd.Series(y_pred_dummy, index=X_test.index)

# Afficher les métriques du DummyClassifier
dummy_unaware_results = report_results(df, y_pred_prob_dummy, y_pred_dummy)
dummy_unaware_results

{'Click Rank Utility': np.float64(0.00853),
 'Negative Log-Likelihood': np.float64(0.23177),
 'AUC': np.float64(0.5),
 'Demographic Parity': np.float64(0.0)}

In [21]:
# Entrainement d'un XGBoost qui ne prend pas en compte l'attribut protégé

import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
import pandas as pd
import numpy as np

X = df.drop(columns=["click", 'protected_attribute'])
y = df["click"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    """ Fonction d'optimisation pour choisir les meilleurs hyperparamètres """
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'objective': 'binary:logistic',
        'eval_metric': 'auc'
    }
    
    model = xgb.XGBClassifier(**params)
    score = cross_val_score(model, X_train, y_train, scoring='roc_auc', cv=3).mean()

    #print(f"Trial {trial.number}: AUC = {score:.5f}, Params = {params}")
    
    return score

# Effectuer 100 essais pour le tuning
N_trials = 50
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=N_trials)

best_params = study.best_params
print("Best parameters:", best_params)

# Entraîner le modèle final avec les meilleurs paramètres
best_model = xgb.XGBClassifier(**best_params)
best_model.fit(X_train, y_train)

y_pred_prob1 = best_model.predict_proba(X_test)[:, 1]
y_pred1 = (y_pred_prob1 >= 0.5).astype(int)

# Transformer les prédictions en Series avec les bons index
y_pred_prob1 = pd.Series(y_pred_prob1, index=X_test.index)
y_pred1 = pd.Series(y_pred1, index=X_test.index)

# Afficher les métriques
xgb_unaware_results = report_results(df, y_pred_prob1, y_pred1)
xgb_unaware_results

[I 2025-02-12 16:25:53,376] A new study created in memory with name: no-name-57d814fc-0a8e-436f-b923-d25b0a0dec73
[I 2025-02-12 16:26:37,180] Trial 0 finished with value: 0.8014781273746093 and parameters: {'max_depth': 8, 'subsample': 0.6762646802660599, 'n_estimators': 232, 'colsample_bytree': 0.644533834586019}. Best is trial 0 with value: 0.8014781273746093.
[I 2025-02-12 16:27:05,556] Trial 1 finished with value: 0.8011640912886371 and parameters: {'max_depth': 9, 'subsample': 0.6854534034252376, 'n_estimators': 129, 'colsample_bytree': 0.9485423779680141}. Best is trial 0 with value: 0.8014781273746093.
[I 2025-02-12 16:27:26,020] Trial 2 finished with value: 0.8187911738779999 and parameters: {'max_depth': 7, 'subsample': 0.941832896425137, 'n_estimators': 178, 'colsample_bytree': 0.6076202852248869}. Best is trial 2 with value: 0.8187911738779999.
[I 2025-02-12 16:27:50,907] Trial 3 finished with value: 0.8116254249199488 and parameters: {'max_depth': 7, 'subsample': 0.89887542

Best parameters: {'max_depth': 3, 'subsample': 0.8721040931502022, 'n_estimators': 149, 'colsample_bytree': 0.5169089489283786}


{'Click Rank Utility': np.float64(0.00853),
 'Negative Log-Likelihood': np.float64(0.03395),
 'AUC': np.float64(0.84448),
 'Demographic Parity': np.float64(-3e-05)}

In [22]:
# Création du DataFrame à partir des résultats
df_results = pd.DataFrame({
    "Model": ["Dummy unaware", "XGB unaware", "XGB unfair"],
    "NLLH ↓": [dummy_unaware_results["Negative Log-Likelihood"], xgb_unaware_results["Negative Log-Likelihood"], xgb_unfair_results["Negative Log-Likelihood"]],
    "AUC ↑": [dummy_unaware_results["AUC"], xgb_unaware_results["AUC"], xgb_unfair_results["AUC"]],
    "DP ↓": [dummy_unaware_results["Demographic Parity"], xgb_unaware_results["Demographic Parity"], xgb_unfair_results["Demographic Parity"]],
    "U ↑": [dummy_unaware_results["Click Rank Utility"], xgb_unaware_results["Click Rank Utility"], xgb_unfair_results["Click Rank Utility"]],
})
print(df_results)

           Model   NLLH ↓    AUC ↑     DP ↓      U ↑
0  Dummy unaware  0.23177  0.50000  0.00000  0.00853
1    XGB unaware  0.03395  0.84448 -0.00003  0.00853
2     XGB unfair  0.03394  0.84400 -0.00002  0.00853


| Model          | NLLH ↓  | AUC ↑  | DP ↓    | U ↑    |
|---------------|--------|--------|--------|--------|
| Dummy unaware | 0.23177 | 0.50000 | 0.00000 | 0.00853 |
| XGB unaware   | 0.03395 | 0.84298 | -0.00000 | 0.00853 |
| XGB unfair    | 0.03394 | 0.84400 | -0.00002 | 0.00853 |


In [28]:
from sklearn.metrics import classification_report
y_pred_prob1 = best_model.predict_proba(X_test)[:, 1]
y_pred1 = (y_pred_prob1 >= 0.5).astype(int)
print(classification_report(y_pred1, y_test))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00    214443
           1       0.00      0.33      0.00         3

    accuracy                           0.99    214446
   macro avg       0.50      0.66      0.50    214446
weighted avg       1.00      0.99      1.00    214446

